In [15]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from ir_measures import *
import pandas as pd

In [17]:
#pt.init()
ensure_pyterrier_is_loaded()
tira = Client()


In [27]:
pt_dataset = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-01-20240426-training")
pt_dataset_jun = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-06-20240418-training")
pt_dataset_aug = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-08-20240418-training")

In [34]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
sce = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-01-20240423-training')
cb = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
rz = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-01-20240423-training')
fusion_jan = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag1'))
fusion_jun = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag6'))
fusion_aug = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag8'))

In [35]:
pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion_jan,
    ],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
        # "3-max-avg",
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
).sort_values(by="nDCG@10", ascending=False)

,name,nDCG(judged_only=True)@10,nDCG(judged_only=True),nDCG@10,nDCG
4,WSum,0.454667,0.480725,0.251512,0.355077
3,RankZephyr,0.455142,0.482076,0.246549,0.352920
1,Sparse Cross-Encoder,0.448109,0.474784,0.220869,0.336719
2,ColBERT,0.450091,0.476047,0.216457,0.330096
0,BM25,0.444411,0.472638,0.199380,0.317285


In [38]:
bm25_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
sce_jun = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-06-20240418-training')
cb_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
rz_jun = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-06-20240418-training')

### Hypothesis Test on June-Dataset.

In [40]:
pt.Experiment(
    [
        bm25_jun,
        sce_jun,
        cb_jun,
        rz_jun,
        fusion_jun,
    ],
    pt_dataset_jun.get_topics('text'),
    pt_dataset_jun.get_qrels(),
    #pd.read_csv("01_qrels_jun.txt"),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
).sort_values(by="nDCG", ascending=False)


,name,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG,nDCG@10,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
3,RankZephyr,0.400839,0.390550,0.294686,0.227905,84.0,86.0,0.615196,84.0,83.0,0.297015,135.0,176.0,3.441785e-01,118.0,120.0,9.810752e-01
4,WSum,0.399975,0.388633,0.292732,0.227834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sparse Cross-Encoder,0.393611,0.381502,0.276560,0.201901,101.0,145.0,0.038635,96.0,149.0,0.034443,111.0,220.0,1.356805e-04,100.0,193.0,1.761921e-05
2,ColBERT,0.388463,0.376752,0.263776,0.183421,106.0,149.0,0.002078,104.0,151.0,0.003803,95.0,236.0,1.201873e-08,84.0,205.0,9.834654e-11
0,BM25,0.387891,0.375125,0.252253,0.165902,95.0,154.0,0.001333,96.0,153.0,0.001136,97.0,234.0,3.387580e-17,74.0,213.0,1.294152e-17


In [41]:
bm25_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
sce_aug = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-08-20240418-training')
cb_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
rz_aug = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-08-20240418-training')

The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License


Download: 2.48MiB [00:00, 19.1MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/ir-benchmarks/longeval-2023-08-20240418-training/fschlatt


### Hypothesis Test on August-Dataset.

In [42]:
hypo_aug = pt.Experiment(
    [
        bm25_aug,
        sce_aug,
        cb_aug,
        rz_aug,
        fusion_aug,
    ],
    pt_dataset_aug.get_topics('text'),
    pt_dataset_aug.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
).sort_values(by="nDCG", ascending=False)

hypo_aug

The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-padua2024/longeval-2023-08-20240418-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 1.02M/1.02M [00:00<00:00, 9.47MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_datasets/ir-lab-padua-2024/longeval-2023-08-20240418-training/


,name,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG,nDCG@10,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
4,WSum,0.306251,0.303571,0.220011,0.179789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RankZephyr,0.306502,0.303919,0.219348,0.177775,219.0,219.0,0.740464,213.0,218.0,0.655888,418.0,576.0,5.159930e-01,343.0,376.0,1.510246e-01
1,Sparse Cross-Encoder,0.305075,0.302068,0.211935,0.168767,329.0,358.0,0.375969,325.0,359.0,0.297648,447.0,654.0,2.137901e-05,379.0,547.0,8.867774e-06
2,ColBERT,0.305006,0.301849,0.205953,0.161361,346.0,384.0,0.412428,342.0,388.0,0.290758,416.0,676.0,3.051411e-11,361.0,546.0,2.981866e-11
0,BM25,0.300187,0.296796,0.191457,0.140883,307.0,409.0,0.000034,307.0,403.0,0.000013,347.0,771.0,2.643216e-34,283.0,656.0,4.964397e-35
